<a href="https://colab.research.google.com/github/mazenhider/-/blob/main/summary_abst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!pip install transformers datasets accelerate torch --quiet


In [3]:
import pandas as pd
from transformers import pipeline
import torch
import os

# مسار ملفاتك داخل الدرايف
base_path = "/content/drive/MyDrive"

train_path = os.path.join(base_path, "train.csv")
test_path = os.path.join(base_path, "test_features.csv")

# قراءة الملفات
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

print("عدد أوراق التدريب:", len(train_df))
print("عدد أوراق الاختبار:", len(test_df))




عدد أوراق التدريب: 1000
عدد أوراق الاختبار: 345


In [4]:
# عدد النصوص الفارغة في أوراق التدريب والاختبار
print("أوراق التدريب الفارغة:", train_df['text'].isna().sum())
print("أوراق الاختبار الفارغة:", test_df['text'].isna().sum())


أوراق التدريب الفارغة: 0
أوراق الاختبار الفارغة: 0


In [5]:
train_df['text'] = train_df['text'].astype(str).str.strip()
test_df['text'] = test_df['text'].astype(str).str.strip()


In [6]:
train_df = train_df[train_df['text'] != ""]
test_df = test_df[test_df['text'] != ""]


In [7]:
device = 0 if torch.cuda.is_available() else -1

summarizer = pipeline(
    "summarization",
    model="facebook/bart-large-cnn",  # نموذج قوي للتلخيص
    device=device
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [8]:
def safe_summarize(text):
    try:
        if pd.isna(text) or len(str(text).strip()) == 0:
            return "No content to summarize."
        text = str(text)[:1000]  # تقليل النص لتجنب مشاكل الطول
        summary = summarizer(
            text,
            max_length=130,
            min_length=30,
            do_sample=False
        )
        return summary[0]['summary_text']
    except Exception as e:
        return "Summary failed."

# تطبيق التلخيص
test_df['abstract'] = test_df['text'].apply(safe_summarize)


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [12]:
submission = test_df[['paper_id', 'abstract']].copy()
submission.columns = ['paper_id', 'summary']  # هنا غيرنا 'abstract' إلى 'summary'

output_path = os.path.join(base_path, "my_submission.csv")
submission.to_csv(output_path, index=False, encoding='utf-8')

print("✅ تم حفظ ملف الإرسال جاهز للمسابقة في:")
print(output_path)


✅ تم حفظ ملف الإرسال جاهز للمسابقة في:
/content/drive/MyDrive/my_submission.csv


In [9]:
submission = test_df[['paper_id', 'abstract']].copy()
submission.columns = ['id', 'abstract']

output_path = os.path.join(base_path, "my_submission.csv")
submission.to_csv(output_path, index=False, encoding='utf-8')

print("✅ تم حفظ ملف الإرسال في:")
print(output_path)


✅ تم حفظ ملف الإرسال في:
/content/drive/MyDrive/my_submission.csv


In [19]:
# إعادة تسمية الأعمدة لتوافق المسابقة
submission = test_df[['paper_id', 'abstract']].copy()  # إذا عندك paper_id موجود
submission.columns = ['paper_id', 'summary']           # هنا غيرنا abstract إلى summary

# حفظ الملف
output_path = os.path.join(base_path, "my_submissionss.csv")
submission.to_csv(output_path, index=False, encoding='utf-8')

print("✅ تم حفظ الملف بالشكل الصحيح للمسابقة في:")
print(output_path)


✅ تم حفظ الملف بالشكل الصحيح للمسابقة في:
/content/drive/MyDrive/my_submissionss.csv


In [13]:
import os
import pandas as pd

# === 1. ربط Google Drive ===
from google.colab import drive
drive.mount('/content/drive')

# === 2. تحديد مسار الحفظ داخل Drive ===
base_path = "/content/drive/MyDrive"  # هنا ممكن تغير المجلد لو تحب
output_path = os.path.join(base_path, "my_submissions.csv")

# === 3. إنشاء ملف الإرسال ===
submission = test_df[['paper_id', 'abstract']].copy()
submission.columns = ['id', 'abstract']

# === 4. حفظ الملف مباشرة في Drive ===
submission.to_csv(output_path, index=False, encoding='utf-8')

print(f"✅ تم حفظ ملف الإرسال مباشرة في Drive هنا:\n{output_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ تم حفظ ملف الإرسال مباشرة في Drive هنا:
/content/drive/MyDrive/my_submissions.csv


In [17]:
import pandas as pd

# مسار ملفك
file_path = "/content/drive/MyDrive/my_submission.csv"

# قراءة الملف
submission = pd.read_csv(file_path)

# عرض أول 5 أسطر
print(submission.head())

# التحقق من وجود أي قيم فارغة
print("\nعدد القيم الفارغة في كل عمود:")
print(submission.isna().sum())


   paper_id                                            summary
0      1000  Gender disparities persist in how men and wome...
1      1001  Tourism performs poorly on sustainable consump...
2      1002  People work for money, status and power, but o...
3      1003  Max Weber's definition of social action as an ...
4      1004  Research indicates that individuals with a str...

عدد القيم الفارغة في كل عمود:
paper_id    0
summary     0
dtype: int64


In [16]:
# عرض أول 5 صفوف من ملف الإرسال
print(test_df[['paper_id', 'abstract']].head())



   paper_id                                           abstract
0      1000  Gender disparities persist in how men and wome...
1      1001  Tourism performs poorly on sustainable consump...
2      1002  People work for money, status and power, but o...
3      1003  Max Weber's definition of social action as an ...
4      1004  Research indicates that individuals with a str...
